In [20]:
#Import librairies
import cv2
import dlib 
import numpy as np
import os 
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import torch
from tqdm import tqdm
from typing import Optional, Callable
from sklearn.metrics import accuracy_score, f1_score
from sklearn.covariance import LedoitWolf
import json
from collections.abc import Mapping
from torchvision.transforms import v2
import pandas as pd
%load_ext autoreload
%autoreload 2
torch.manual_seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
file_name = r"./coin-detector-model/coin-detector-2.svm"
detector = dlib.simple_object_detector(file_name)

In [22]:
possible_value = ["5CHF", "2CHF", "1CHF", "0.5CHF", "0.2CHF", "0.1CHF", "0.05CHF", "2EUR", "1EUR", "0.5EUR", "0.2EUR", "0.1EUR", "0.05EUR", "0.02EUR", "0.01EUR", "OOD"]
test_folder = r"./data/test"
valid_extensions = ('.jpg')
device = torch.device("gpu" if torch.cuda.is_available() else "cpu")
model = torch.load(r"./model/best_model.pth", map_location=device)
model.eval()

file_list = os.listdir(test_folder)
image_files = [f for f in file_list if f.lower().endswith(valid_extensions)]

final_count = []

image_files = sorted(image_files)

for image_file in tqdm(image_files):
    answer = {}
    coins = []
    image_path = os.path.join(test_folder, image_file)
    image = cv2.imread(image_path)
    image_name = image_file.removesuffix(".JPG")
    if image is None:
        print(f"Failed to load {image_file}")
        continue
    
    detected_im = detector(image)
    for i, detect_test in enumerate(detected_im):
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        d_box = detect_test
        coins.append(img_rgb[d_box.top():d_box.bottom(), d_box.left():d_box.right()])
    coin_count = {}
    answer["id"] = image_name
    for i in range(16):
        answer[possible_value[i]] = 0

    for c in coins:
        if(c.shape[0] == 0 or c.shape[1] == 0):
            continue
        c = torch.tensor(c, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
        transform = v2.Compose([
            v2.Resize(224),
            v2.CenterCrop(224),
            v2.ToTensor(),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        with torch.no_grad():
            c = transform(c)
            output = model(c)
            res = output.argmax()
            answer[possible_value[res.item()]] += 1
    final_count.append(answer)

print("Creating csv file...")
print(final_count)
pandas_df = pd.DataFrame(final_count)
pandas_df.to_csv(r"./data/test-predictions.csv", index=False)



  0%|          | 0/81 [00:00<?, ?it/s]/home/bluedrack/.conda/envs/iapr/lib/python3.9/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
100%|██████████| 81/81 [03:42<00:00,  2.74s/it]

Creating csv file...
[{'id': 'L1010277', '5CHF': 0, '2CHF': 0, '1CHF': 0, '0.5CHF': 2, '0.2CHF': 0, '0.1CHF': 0, '0.05CHF': 0, '2EUR': 1, '1EUR': 0, '0.5EUR': 1, '0.2EUR': 0, '0.1EUR': 0, '0.05EUR': 0, '0.02EUR': 0, '0.01EUR': 0, 'OOD': 1}, {'id': 'L1010279', '5CHF': 0, '2CHF': 0, '1CHF': 0, '0.5CHF': 1, '0.2CHF': 0, '0.1CHF': 0, '0.05CHF': 0, '2EUR': 0, '1EUR': 0, '0.5EUR': 0, '0.2EUR': 0, '0.1EUR': 0, '0.05EUR': 0, '0.02EUR': 0, '0.01EUR': 0, 'OOD': 3}, {'id': 'L1010281', '5CHF': 0, '2CHF': 0, '1CHF': 0, '0.5CHF': 1, '0.2CHF': 0, '0.1CHF': 0, '0.05CHF': 0, '2EUR': 0, '1EUR': 0, '0.5EUR': 1, '0.2EUR': 0, '0.1EUR': 0, '0.05EUR': 0, '0.02EUR': 0, '0.01EUR': 0, 'OOD': 6}, {'id': 'L1010283', '5CHF': 0, '2CHF': 0, '1CHF': 0, '0.5CHF': 0, '0.2CHF': 0, '0.1CHF': 0, '0.05CHF': 0, '2EUR': 0, '1EUR': 0, '0.5EUR': 1, '0.2EUR': 0, '0.1EUR': 0, '0.05EUR': 0, '0.02EUR': 0, '0.01EUR': 0, 'OOD': 5}, {'id': 'L1010287', '5CHF': 0, '2CHF': 0, '1CHF': 0, '0.5CHF': 2, '0.2CHF': 0, '0.1CHF': 0, '0.05CHF': 

In [23]:
# Compare prediction with labels
labels = pd.read_csv(r"./data/train_labels.csv")
#labels = labels.sort_values(by="id")
predictions = pd.read_csv(r"./data/train-predictions.csv")
#predictions = predictions.sort_values(by="id")
diff = labels.compare(predictions)
print(diff)

          id           5CHF       2CHF       1CHF       0.5CHF        ...  \
        self     other self other self other self other   self other  ...   
0   L1010281  L1010277  NaN   NaN  NaN   NaN  NaN   NaN    0.0   2.0  ...   
1   L1010298  L1010279  NaN   NaN  2.0   0.0  1.0   0.0    2.0   1.0  ...   
2   L1010321  L1010281  NaN   NaN  NaN   NaN  1.0   0.0    2.0   1.0  ...   
3   L1010310  L1010283  NaN   NaN  NaN   NaN  1.0   0.0    1.0   0.0  ...   
4   L1010300  L1010287  NaN   NaN  1.0   0.0  NaN   NaN    0.0   2.0  ...   
..       ...       ...  ...   ...  ...   ...  ...   ...    ...   ...  ...   
76  L1010524  L1010515  1.0   0.0  NaN   NaN  NaN   NaN    NaN   NaN  ...   
77  L1010506  L1010517  1.0   0.0  NaN   NaN  NaN   NaN    0.0   1.0  ...   
78       NaN       NaN  1.0   0.0  NaN   NaN  NaN   NaN    0.0   1.0  ...   
79  L1010515  L1010523  NaN   NaN  NaN   NaN  NaN   NaN    NaN   NaN  ...   
80  L1010523  L1010524  NaN   NaN  NaN   NaN  NaN   NaN    NaN   NaN  ...   